In [5]:
import subprocess
import os
import signal
import py3nvml.py3nvml as nvml

def install_packages():
    subprocess.run(["pip", "install", "py3nvml"], check=True)

def get_gpu_processes():
    nvml.nvmlInit()
    device_count = nvml.nvmlDeviceGetCount()
    gpu_processes = []
    for i in range(device_count):
        handle = nvml.nvmlDeviceGetHandleByIndex(i)
        procs = nvml.nvmlDeviceGetComputeRunningProcesses(handle)
        for proc in procs:
            print(f"Device {i}: PID {proc.pid}, Memory {proc.usedGpuMemory // 1024 ** 2} MB")
            gpu_processes.append(proc.pid)
    nvml.nvmlShutdown()
    return gpu_processes

def kill_gpu_processes(pids):
    for pid in pids:
        try:
            os.kill(pid, signal.SIGTERM)  # Trying SIGTERM first
            print(f"Sent SIGTERM to PID {pid}")
        except OSError as e:
            print(f"Failed to send SIGTERM to PID {pid}: {str(e)}")
            # Check if process exists using ps
            ps_result = subprocess.run(["ps", "-p", str(pid), "-o", "comm="], capture_output=True, text=True)
            if ps_result.stdout:
                print(f"Process {pid} still running: {ps_result.stdout.strip()}")
            else:
                print(f"Process {pid} not found running.")

def main():
    print("Installing packages...")
    install_packages()
    print("Fetching GPU processes...")
    gpu_pids = get_gpu_processes()
    print("Attempting to kill GPU processes...")
    kill_gpu_processes(gpu_pids)

main()


Installing packages...
Fetching GPU processes...
Device 0: PID 1937399, Memory 13078 MB
Device 0: PID 1939529, Memory 11540 MB
Attempting to kill GPU processes...
Failed to send SIGTERM to PID 1937399: [Errno 3] No such process
Process 1937399 not found running.
Failed to send SIGTERM to PID 1939529: [Errno 3] No such process
Process 1939529 not found running.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
